In [1]:
from __future__ import print_function, division
from data_utils import load_CIFAR10

X_train, y_train, X_test, y_test = load_CIFAR10("cifar-10-batches-py")

print("X_train:", X_train.shape)
print("y_train:", y_train.shape)
print("X_test:", X_test.shape)
print("y_test:", y_test.shape)

X_train: (50000, 32, 32, 3)
y_train: (50000,)
X_test: (10000, 32, 32, 3)
y_test: (10000,)


In [5]:
orie = 9
ppc = (6,6)
cpb = (3,3)

import numpy as np
from skimage.feature import hog
from skimage import color, exposure

tmp_train = []
for i in xrange(X_train.shape[0]):
    gray_img = color.rgb2gray(X_train[i])
    fd = hog(gray_img, orientations=orie, pixels_per_cell=ppc,
                    cells_per_block=cpb, visualise=False, transform_sqrt=True)
    fd = fd[np.newaxis, :]
    tmp_train.append(fd)
X_train_hog = np.concatenate(tmp_train, axis=0)

tmp_test = []
for i in xrange(X_test.shape[0]):
    gray_img = color.rgb2gray(X_test[i])
    fd = hog(gray_img, orientations=orie, pixels_per_cell=ppc,
                    cells_per_block=cpb, visualise=False, transform_sqrt=True)
    fd = fd[np.newaxis, :]
    tmp_test.append(fd)

X_test_hog = np.concatenate(tmp_test, axis=0)

print(X_train_hog.shape)
print(X_test_hog.shape)

(50000, 729)
(10000, 729)


In [6]:
from sklearn.ensemble import BaggingClassifier
from sklearn import svm
import time
bag_clf = BaggingClassifier(base_estimator=svm.LinearSVC(), n_estimators=10, max_samples=0.5, max_features=0.5, n_jobs=-1)

tic = time.time()
bag_clf.fit(X_train_hog, y_train)
toc_1 = time.time()
print("time for training: %d seconds" %(toc_1-tic))

preds = bag_clf.predict(X_test_hog)
toc_2 = time.time()
print("time for prediction: %d seconds" %(toc_2-toc_1))

acc = (preds==y_test).mean()
print("accuracy: ", acc)


time for training: 472 seconds
time for prediction: 1 seconds
accuracy:  0.516


In [8]:
preds = bag_clf.predict(X_train_hog)

acc = (preds==y_train).mean()
print("accuracy: ", acc)

accuracy:  0.53352
